Fetching Webpages with the Requests Library To Demonstrate
===========================================
SqlAlchemy helps you use a database to store and retrieve information from python.  It abstracts the specific storage engine from te way you use it - so it doesn't care if you end up using MySQL, SQLite, or whatever else. In addition, you can use core and the object-relational mapper (ORM) to avoid writing any SQL at all.  The [SQLAlchemy homepage](http://www.sqlalchemy.org/) has lots of good examples and full documentation.

In [6]:
from sqlalchemy import *
import datetime
import mediacloud

In [7]:
my_api_key = 'your api key here'
mc = mediacloud.api.MediaCloud(my_api_key)

## Basic SQL Generation
The core library generates SQL for you.  Read more about it on their [expression language tutorial page](http://docs.sqlalchemy.org/en/rel_1_0/core/index.html). Below are some basic examples.

### Creating a Table
Read more about [defining and creating tables](http://docs.sqlalchemy.org/en/rel_1_0/core/tutorial.html#define-and-create-tables).

In [8]:
# add `echo=True` to see log statements of all the SQL that is generated
engine = create_engine('sqlite:///:memory:') # just save the db in memory for now (ie. not on disk)
metadata = MetaData()

def getMediaCloudCount(keyword):
    return mc.sentenceCount('('+keyword+')', solr_filter=[mc.publish_date_query( 
                datetime.date( 2014, 1, 1), datetime.date( 2015, 1, 1) ), 'media_sets_id:1' ])['count']

# define a table to use
queries = Table('queries', metadata,
    Column('id', Integer, primary_key=True),
    Column('keywords', String(400), nullable=False),
    Column('timestamp', DateTime, default=datetime.datetime.now),
    Column('count', Integer),
)
metadata.create_all(engine) # and create the tables in the database

### Inserting Data
Read more about generating [SQL insert statements](http://docs.sqlalchemy.org/en/rel_1_0/core/tutorial.html#insert-expressions).

In [9]:
insert_stmt = queries.insert()
str(insert_stmt) # see an example of what this will do

'INSERT INTO queries (id, keywords, timestamp, count) VALUES (:id, :keywords, :timestamp, :count)'

In [10]:
insert_stmt = queries.insert().values(keywords="puppies", count=getMediaCloudCount("puppies"))
str(insert_stmt)

'INSERT INTO queries (keywords, timestamp, count) VALUES (:keywords, :timestamp, :count)'

In [11]:
db_conn = engine.connect()
result = db_conn.execute(insert_stmt)
result.inserted_primary_key # print out the primary key it was assigned

[1]

In [12]:
insert_stmt = queries.insert().values(keywords="kittens", count=getMediaCloudCount("kittens"))
result = db_conn.execute(insert_stmt)
result.inserted_primary_key # print out the primary key it was assigned

[2]

### Retrieving Data
Read more about using [SQL select statments](http://docs.sqlalchemy.org/en/rel_1_0/core/tutorial.html#selecting).

In [13]:
from sqlalchemy.sql import select
select_stmt = select([queries])
results = db_conn.execute(select_stmt)
for row in results:
    print row

(1, u'puppies', datetime.datetime(2015, 12, 8, 19, 16, 7, 430254), 5709)
(2, u'kittens', datetime.datetime(2015, 12, 8, 19, 16, 10, 822920), 3101)


In [14]:
select_stmt = select([queries]).where(queries.c.id==1)
for row in db_conn.execute(select_stmt):
    print row

(1, u'puppies', datetime.datetime(2015, 12, 8, 19, 16, 7, 430254), 5709)


In [15]:
select_stmt = select([queries]).where(queries.c.keywords.like('p%'))
for row in db_conn.execute(select_stmt):
    print row

(1, u'puppies', datetime.datetime(2015, 12, 8, 19, 16, 7, 430254), 5709)


In [16]:
counting = 0
select_stmt = select([queries])
results = db_conn.execute(select_stmt)
for row in results:
    counting+=row['count']

print counting

8810


## ORM
You can use their ORM library to handle the translation into full-fledged python objects.  This can help you build the Model for you [MVC](https://en.wikipedia.org/wiki/Model–view–controller) solution.

In [17]:
import datetime
from sqlalchemy import *
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.hybrid import hybrid_property
Base = declarative_base()

### Creating a class mapping
Read more about [creating a mapping](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#declare-a-mapping).

In [18]:
class Query(Base):
    __tablename__ = 'queries'
    id = Column(Integer, primary_key=True)
    keywords = Column(String(400))
    timestamp = Column(DateTime,default=datetime.datetime.now)
    count = Column(Integer)
    
    @hybrid_property
    def count(self):
        return getMediaCloudCount(self.keywords)

    def __repr__(self):
        return "<Query(keywords='%s' count='%d')>" % (self.keywords, self.count)
Query.__table__

Table('queries', MetaData(bind=None), Column('id', Integer(), table=<queries>, primary_key=True, nullable=False), Column('keywords', String(length=400), table=<queries>), Column('timestamp', DateTime(), table=<queries>, default=ColumnDefault(<function <lambda> at 0x108d38668>)), schema=None)

### Creating a connection and session
Read more about [creating this stuff](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#creating-a-session).

In [19]:
engine = create_engine('sqlite:///:memory:') # just save the db in memory for now (ie. not on disk)
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
my_session = Session()

### Inserting Data
Read more about [inserting data with an ORM](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#adding-new-objects).

In [20]:
query = Query(keywords="iguana")
query.keywords

'iguana'

In [21]:
my_session.add(query)
my_session.commit()
print query.id

1


### Retrieving Data
Read more about [retrieving data from the db](http://docs.sqlalchemy.org/en/rel_1_0/orm/tutorial.html#querying) via an ORM class.

In [22]:
for q in my_session.query(Query).order_by(Query.timestamp):
    print q

<Query(keywords='iguana' count='459')>


In [23]:
query1 = Query(keywords="robot")
query2 = Query(keywords="puppy")
my_session.add_all([query1,query2])
my_session.commit()

In [24]:
for q in my_session.query(Query).order_by(Query.timestamp):
    print q

<Query(keywords='iguana' count='459')>
<Query(keywords='robot' count='10340')>
<Query(keywords='puppy' count='11042')>


In [25]:
for q in my_session.query(Query).filter(Query.keywords.like('p%')):
    print q

<Query(keywords='puppy' count='11042')>


In [26]:
sentenceCount=0;
for q in my_session.query(Query).all():
    sentenceCount+=q.count
    
print sentenceCount
    

21841
